# ETL of Gmail Data
## ABB #1 - Session 2

Code authored by: Shaw Talebi

### imports

In [1]:
import imaplib
import email
from email.policy import default
import pandas as pd

from top_secret import app_password

### functions

In [2]:
def parse_emails(mail, mailbox_name):
    """
        Parse emails from a given mailbox and return a dataframe with the emails and labels.
    """

    # Step 2: Select the folder
    mail.select(mailbox_name)  # 'inbox' or 'spam'

    # Step 3: Search for emails
    _, messages = mail.search(None, "ALL")

    # Step 4: Extract emails
    # Fetch and parse the emails
    email_data = []
    for msg_id in messages[0].split():
        _, msg_data = mail.fetch(msg_id, "(RFC822)")
        
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1], policy=default)
                subject = msg["subject"]
                from_ = msg["from"]
                if msg.is_multipart():
                    body = ""
                    for part in msg.walk():
                        if part.get_content_type() == "text/plain":
                            body += part.get_payload(decode=True).decode('utf-8', errors='ignore')
                else:
                    body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')

                email_data.append({
                    "subject": subject,
                    "from": from_,
                    "body": body
                })

    return email_data

In [3]:
def extract_folder_names(mail, keyword):
    """
        Extract folder names containing a given keyword.
    """
    # Get list of folders
    _, folders = mail.list()

    # Extract folder names containing "Personal"
    folder_list = []
    for folder in folders:
        # Decode from bytes to string and split by quotes
        folder_str = folder.decode('utf-8')
        # Extract folder name (it's between the last quote marks)
        folder_name = folder_str.split('"')[-2]
        
        if keyword in folder_name:
            folder_list.append(folder_name)

    return folder_list

In [4]:
def parse_emails_from_keyword(mail, num_emails, keyword, label):
    """
        Parse emails from a given keyword and return a dataframe with the emails and labels.
    """

    folder_list = extract_folder_names(mail, keyword)

    email_data = []
    for mailbox_name in folder_list:
        try:
            email_data = email_data + parse_emails(mail, mailbox_name)
        except:
            print(f"Could not parse {mailbox_name}")

        if len(email_data) > num_emails:
            break

    df = pd.DataFrame(email_data)
    df["label"] = label
    
    return df

### Extract

In [5]:
# define email details
imap_server = "imap.gmail.com"
email_address = "your email here"

# establish connection to the server
mail = imaplib.IMAP4_SSL(imap_server)
mail.login(email_address, app_password)

('OK', [b'swcballa@gmail.com authenticated (Success)'])

In [6]:
# extract personal emails
num_emails = 150
keyword = "Personal"
label = "personal"

df_personal = parse_emails_from_keyword(mail, num_emails, keyword, label)
print(len(df_personal))

Could not parse Personal/Family/Amoo Ali
254


In [7]:
# extract non-personal emails
keyword = "STV"
label = "not personal"

df_not_personal = parse_emails_from_keyword(mail, num_emails, keyword, label)
print(len(df_not_personal))

Could not parse STV/Amazon Assoc
160


In [8]:
# log out of the server
mail.logout()

('BYE', [b'LOGOUT Requested'])

### Transform

In [9]:
# merge emails into a single dataframe
df = pd.concat([df_personal[:num_emails], df_not_personal[:num_emails]])
print(len(df))

300


In [10]:
# feature engineering

# length of the body
df["body_length"] = df["body"].apply(len)

# contains "lol"
df["contains_lol"] = df["body"].apply(lambda x: "lol" in x.lower())

# contains "OMG"
df["contains_omg"] = df["body"].apply(lambda x: "omg" in x.lower())

# contains "attached" or "see attached" or "see attachment"
df["contains_attached"] = df["body"].apply(lambda x: "attached" in x.lower())
df["contains_attachment"] = df["body"].apply(lambda x: "attachment" in x.lower())

# contains "Order Confirmation"
df["contains_order_confirmation"] = df["body"].apply(lambda x: "order confirmation" in x.lower())

# contains "payment summary"
df["contains_payment_summary"] = df["body"].apply(lambda x: "payment summary" in x.lower())

# sender is common person domain (gmail, yahoo, hotmail) 
df["sender_has_common_domain"] = df["from"].apply(lambda x: any(domain in x.lower() for domain in ["gmail", "yahoo", "hotmail"]))

# is personal email
df["is_personal"] = df["label"] == "personal"

In [11]:
# drop original columns
df_transformed = df.iloc[:, 4:]
df_transformed.head()

,body_length,contains_lol,contains_omg,contains_attached,contains_attachment,contains_order_confirmation,contains_payment_summary,sender_has_common_domain,is_personal
0,1830,False,False,False,False,False,False,False,True
1,1522,False,False,False,False,False,False,True,True
2,1838,False,False,False,False,False,False,False,True
3,251,False,False,False,True,False,False,False,True
4,3170,False,False,False,False,False,False,False,True


### Load

In [12]:
# save to csv
df_transformed.to_csv("data/transformed_data.csv", index=False)